[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_returns_distribution/EMQA_returns_distribution.ipynb)

# EMQA_returns_distribution

Fat tails in energy return distributions.

**Output:** `lecture1_returns_distribution.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from scipy import stats as sp_stats

brent = fetch('BZ=F'); gas = fetch('NG=F')
bret = np.log(brent/brent.shift(1)).dropna()
gret = np.log(gas/gas.shift(1)).dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for ax, r, name, c in [(axes[0], bret, 'Brent Oil', COLORS['blue']),
                         (axes[1], gret, 'Natural Gas', COLORS['green'])]:
    ax.hist(r*100, bins=80, density=True, color=c, alpha=0.6, edgecolor='white')
    x = np.linspace(r.min()*100, r.max()*100, 300)
    ax.plot(x, sp_stats.norm.pdf(x, r.mean()*100, r.std()*100), color=COLORS['red'], lw=2)
    kurt = sp_stats.kurtosis(r, fisher=False)
    ax.set_title(f'{name} Returns (Kurtosis: {kurt:.1f})', fontweight='bold')
    ax.set_xlabel('Daily Return (%)')

fig.legend(['Normal Distribution', 'Actual Returns'], loc='upper center',
           bbox_to_anchor=(0.5, -0.03), ncol=2, frameon=False)
fig.tight_layout()
save_fig(fig, 'lecture1_returns_distribution.pdf')
plt.show()
